# Expected Output

In [1]:
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import convolve
import pickle
import os
from common_functions import *
import folium

# Constants

In [2]:
BASE_ROI_FOLDER = 'regions'
MANGROVE_GEOJSON = 'colombia_gmw.geojson'

In [3]:
PERCENTILE_THRESHOLD = 90

# Driver Code

In [4]:
training_data = pickle.load(open('stored_training_data.p', 'rb'))
regions_of_interest = list(training_data['training_data'].keys())

In [5]:
wetland_type = training_data['aux_info']['wetland_type']

In [6]:
colMap = folium.Map(location=[3, -74], tiles='Stamen Terrain', zoom_start=6)
predicted_group = folium.FeatureGroup('predicted wetlands')
baseline_group = folium.FeatureGroup('baseline wetlands')

In [7]:
for folder in regions_of_interest:
    
    print(folder)
    
    #get names of predicted wetlands and original baseline rasters
    predicted_baseline_fname = '%s/%s/predicted_%s.tiff'%(BASE_ROI_FOLDER, folder, folder)
    baseline_fname = '%s/%s/baseline_%s_%s.tiff'%(BASE_ROI_FOLDER, folder, wetland_type.replace('/','').replace(' ',''), folder)

    #open the map
    ds = gdal.Open(predicted_baseline_fname, gdal.GA_ReadOnly)
    arr = ds.ReadAsArray()
    gt = ds.GetGeoTransform()
    ds = None
    
    
    #get the corresponding baseline array
    ds = gdal.Open(baseline_fname, gdal.GA_ReadOnly)
    baseline_arr = ds.ReadAsArray()
    scale = arr.shape[0] // baseline_arr.shape[0]
    baseline_arr = np.repeat(np.repeat(baseline_arr, scale, axis=0), scale, axis=1)
    
    ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()
    
    lrx = ulx + (ds.RasterXSize * xres)
    lry = uly + (ds.RasterYSize * yres)
    
    folium.raster_layers.ImageOverlay(
        image=baseline_arr,
        bounds=[[lry, ulx], [uly, lrx]],
        opacity=0.5,
        interactive=True,
        cross_origin=False,
        zindex=1,
    ).add_to(baseline_group)
    
    ds = None

    #get the data mask
    data_mask = (baseline_arr != -1)&(arr != -1)
    
    #only show predicted areas which are kth percentile and above (confident wetland predictions)
    thresh = np.percentile(arr[arr!=-1], PERCENTILE_THRESHOLD)

    #difference between predicted and baseline
    new_wetlands = (arr - baseline_arr > thresh)
    prev_wetlands = (arr - baseline_arr < -thresh)
    
    #convolve these differences with low pass filter to remove noise
    size = 3
    kernel = np.ones((size,size)) / size**2
    for w in [new_wetlands, prev_wetlands]:
        b = convolve(w, kernel, mode='same')
        w[abs(b) < thresh] = 0
        w[~data_mask] = 0
    
    #create a new baseline by adding the original baseline to the differences (novel wetlands) and subtracting previous wetlands
    new_baseline = baseline_arr + new_wetlands - prev_wetlands
    shown_baseline = new_baseline.copy()
    shown_baseline[new_baseline == -1] = 0
    
    #save the new baseline to the respective sub-directory
    new_baseline_fname = '%s/%s/new_baseline_%s_%s.tiff'%(BASE_ROI_FOLDER, folder, wetland_type.replace('/','').replace(' ',''), folder)
    ds = np_array_to_raster(new_baseline_fname, new_baseline, gt, no_data=-1, nband=1, gdal_data_type=gdal.GDT_Float64)
    
    #ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()
    #lrx = ulx + (ds.RasterXSize * xres)
    #lry = uly + (ds.RasterYSize * yres)
    #print([[lry, ulx], [uly, lrx]])
    
    folium.raster_layers.ImageOverlay(
        image=shown_baseline,
        bounds=[[lry, ulx], [uly, lrx]],
        opacity=0.5,
        interactive=True,
        cross_origin=False,
        zindex=1,
    ).add_to(predicted_group)
    
    ds = None

#if we are predicting mangroves, try to add the GMW layer to the generated map
if wetland_type == 'mangrove':
    try:
        folium.GeoJson(
            MANGROVE_GEOJSON,
            name='Global Mangrove Watch'
        ).add_to(colMap)
    except:
        pass

predicted_group.add_to(colMap)
baseline_group.add_to(colMap)
folium.LayerControl().add_to(colMap)

colMap.save('map.html')

region_0_0_0
region_0_0_1
region_0_1_0
region_0_1_1
region_1_0_0
region_2_0_0
region_2_0_1
region_2_0_2
region_3_0_0
region_3_0_1
region_3_1_0
region_3_1_1
